In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy

In [3]:
def get_title(soup):
    try:
        title=soup.find("span",attrs={"id":"productTitle"}).text.strip()
        return title
    except AttributeError:
        title=""
    return title

In [4]:
def get_price(soup):
    try:
        price=soup.find("span",attrs={"class":"a-offscreen"}).text
        return price
    except AttributeError:
         price=""
    return price

In [5]:
def get_rating(soup):
    try:
        rating=soup.find("span",attrs={"class":"a-icon-alt"}).text
        return rating
    except AttributeError:
         rating=""
    return rating

In [6]:
def get_ratingno(soup):
    try:
        ratingno=soup.find("span",attrs={"id":"acrCustomerReviewText"}).text
        return ratingno
    except AttributeError:
         ratingno=""
    return ratingno

In [7]:
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	
    return available

In [8]:
if __name__ == '__main__':
    URL = "https://www.amazon.com/s?k=shoes&crid=14O13XAKFJ74S&sprefix=shoes%2Caps%2C177&ref=nb_sb_noss_2"
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    webpage = requests.get(URL,headers=HEADERS)
    soup = BeautifulSoup(webpage.content,"html.parser")
    links = soup.find_all("a",attrs={'class':'a-link-normal s-no-outline'})
    links_list=[];
    for link in links:
        links_list.append(link.get('href'))
    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    for link in links_list:
        product_link="https://www.amazon.com/" + link
        new_webpage = requests.get(product_link,headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content,"html.parser")
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_ratingno(new_soup))
        d['availability'].append(get_availability(new_soup))
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', numpy.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [9]:
amazon_df

,title,price,rating,reviews,availability
0,Under Armour Men's Charged Assert 9 Running Shoe,$49.95,4.6 out of 5 stars,"52,071 ratings",Not Available
1,Steve Madden Mens Fenta Shoes,$46.07,4.3 out of 5 stars,"5,996 ratings",Not Available
2,Bruno Marc Men's Rivera Oxfords Shoes Sneakers,$35.99,4.2 out of 5 stars,"9,715 ratings",Not Available
3,K-Swiss Men's Tubes 200 Training Shoe,$29.90,4.6 out of 5 stars,"3,082 ratings",Not Available
4,Merrell Men's Moab 3 Hiking Shoe,$49.38,4.6 out of 5 stars,"3,972 ratings",Not Available
...,...,...,...,...,...
65,WONESION Mens Breathable Walking Tennis Runnin...,$45.99,4.1 out of 5 stars,"21,220 ratings",Only 4 left in stock - order soon
66,Pujcs Womens Sneakers Slip On Walking Shoes Br...,$29.99,4.3 out of 5 stars,217 ratings,In Stock
67,SKOKOD Steel Toe Shoes for Men Puncture Proof ...,$42.99,4.1 out of 5 stars,50 ratings,In Stock
68,Brooks Men’s Adrenaline GTS 23 Supportive Runn...,$139.95,4.8 out of 5 stars,156 ratings,Only 4 left in stock - order soon
